In [1]:
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

/home/kevin/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_json('./input/train.json')
test = pd.read_json("./input/test.json")

from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(\
    train[['bathrooms', 'bedrooms','price','latitude','longitude']], \
    train[['interest_level']].values, \
    test_size=0.33, random_state=42)

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train[:,0])
y_train_encoded = le.transform(y_train[:,0])
y_validate_encoded = le.transform(y_validate[:,0])

In [4]:
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)

In [5]:
dvalidate = xgb.DMatrix(X_validate, label=y_validate_encoded)

In [6]:
evallist  = [(dvalidate,'eval'), (dtrain,'train')]

In [7]:
param = {'bst:max_depth':6, \
         'bst:eta':0.3, \
         'silent':1,
         'objective':'multi:softprob',
         'num_class':3}

num_round = 70
bst = xgb.train(param, dtrain, num_round, evallist )

[0]	eval-merror:0.300731	train-merror:0.288402
[1]	eval-merror:0.298275	train-merror:0.28568
[2]	eval-merror:0.29809	train-merror:0.284772
[3]	eval-merror:0.299257	train-merror:0.285438
[4]	eval-merror:0.297722	train-merror:0.284863
[5]	eval-merror:0.29588	train-merror:0.283623
[6]	eval-merror:0.295635	train-merror:0.283109
[7]	eval-merror:0.294775	train-merror:0.282232
[8]	eval-merror:0.293731	train-merror:0.28075
[9]	eval-merror:0.293179	train-merror:0.279903
[10]	eval-merror:0.29281	train-merror:0.279087
[11]	eval-merror:0.292933	train-merror:0.278693
[12]	eval-merror:0.292749	train-merror:0.278331
[13]	eval-merror:0.293056	train-merror:0.277574
[14]	eval-merror:0.292196	train-merror:0.276425
[15]	eval-merror:0.293056	train-merror:0.276032
[16]	eval-merror:0.292687	train-merror:0.275034
[17]	eval-merror:0.292135	train-merror:0.273643
[18]	eval-merror:0.291705	train-merror:0.273038
[19]	eval-merror:0.291766	train-merror:0.272433
[20]	eval-merror:0.291214	train-merror:0.271919
[21]	ev

In [8]:
predict_validate = bst.predict(dvalidate)

In [9]:
from sklearn.metrics import log_loss
print(log_loss(y_validate, predict_validate))

0.65806997973


In [10]:
dtest = xgb.DMatrix(test[['bathrooms', 'bedrooms','price','latitude','longitude']])
test_predict = bst.predict(dtest)

In [11]:
test_predict

array([[ 0.02696094,  0.67625225,  0.29678684],
       [ 0.3333098 ,  0.32053104,  0.34615916],
       [ 0.01207564,  0.86215872,  0.12576565],
       ..., 
       [ 0.1124459 ,  0.65860081,  0.22895329],
       [ 0.21451271,  0.36508676,  0.42040053],
       [ 0.03242121,  0.7406857 ,  0.22689304]], dtype=float32)

In [12]:
result_frame = pd.DataFrame(test_predict)
result_frame.columns = le.classes_
result_frame['listing_id'] = test.listing_id.reset_index().listing_id
result_frame

,high,low,medium,listing_id
0,0.026961,0.676252,0.296787,7142618
1,0.333310,0.320531,0.346159,7210040
2,0.012076,0.862159,0.125766,7103890
3,0.028167,0.795567,0.176267,7143442
4,0.011822,0.908928,0.079250,6860601
5,0.000600,0.972590,0.026811,6840081
6,0.152640,0.570057,0.277303,6922337
7,0.170513,0.424610,0.404877,6913616
8,0.147630,0.443885,0.408485,6937820
9,0.082166,0.675339,0.242495,6893933


In [14]:
# Not submitted
result_frame[['listing_id','high','medium','low']].to_csv('XGB-result.csv', \
                                                          index = False)

0.66043